In [1]:
import time
import numpy as np
import cv2

# CrazyFlie imports:

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander

In [2]:
camera = 0
group_number = 9

In [3]:
# Sort through contours in the image
def findGreatesContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index

In [4]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # crop the frame: take 100 off each side x values
    frame = frame[:, 100:540]
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (158, 50, 70)
    ub1 = (180, 255, 255)
    lb2 = (0, 50, 70)
    ub2 = (10, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if not contours:
        continue
    largest_area, largest_contour_index = findGreatesContour(contours)
    largest = contours[largest_contour_index]
    
    #print(largest_area)
    # will only show the bounding box when we are showing the FRAME not the MASK
    # this is because the mask is a 2d image NOT a 3d image, so it cannot show complex colors such as green
    # will only show gray, which we cannot see
    x, y, w, h = cv2.boundingRect(largest)
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    center_x = 250
    rect_center = x+(w/2)
    
    diff = center_x - rect_center
    
    if (rect_center > center_x -50) and (rect_center < center_x + 50):
        #print(diff)
        front = True
        
    #else: print('False')
    
    #center_x = 220
    #diff_center = 7s0
    
    #if (x > center_x - diff_center) and (x + w < center_x + diff_center):
    #    front = True
    #else: front = False 
    #print(front)
    # Compute
    cv2.imshow('mask', frame)
    
    #print(frame.shape)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

## Helper Functions

The following cell contains some sample functions which will be useful.

In particular, __check_contours__ and __findGreatesContour__ will perform red filtering on the live camera feed and identify the obstacles. The red filtering is controlled by setting HSV intervals in the __check_contours__ function. Note that the intervals will require tuning and may vary on different drones/cameras.

The __adjust_position__ function can also be modified for performing obstacle avoidance.

In [12]:
# Get the current crazyflie position:
def position_estimate(scf):
    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]
            x = data['kalman.varPX']
            y = data['kalman.varPY']
            z = data['kalman.varPZ']
            
    print(x, y, z)
    return x, y, z


# Set the built-in PID controller:
def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    time.sleep(2)
    return


# Ascend and hover at 1m:
def ascend_and_hover(cf):
    # Ascend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, y / 10)
        time.sleep(0.1)
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1.0)
        time.sleep(0.1)
    return

def hover(cf):
    print("Hovering...")
    # Hover at 0.5 meters:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, 1.0)
        time.sleep(0.1)
    return
    
# find mask
def get_mask(frame):
    
    # These define the upper and lower HSV for the red obstacles.
    # Note that the red color wraps around 180, so there are two intervals.
    # Tuning of these values will vary depending on the camera.
    lb1 = (158, 50, 70)
    ub1 = (180, 255, 255)
    lb2 = (0, 50, 70)
    ub2 = (10, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)
    
    return mask


def check_contours(mask, diff_center, max_area=10000):
    """ 
    Check if there is an obstacle right in front of us.
    1. find the largest obstacle (this should be the closest)
    2. check if it within a range from the center of the image
        This will check for obstacle only in front of us, not just in our field of view
        
    mask: black and white mask: white signifying obstacles
    diff_center: number of pixels away from center the obstacle can be
        i.e. if 20, then checks for obstacle in [img_center_x-20, im_center_x+20]
    max_area: hyper param to be tuned through trials: find out what the actual largest area we want is
    """
    

    print('Checking image:')
    # Use the OpenCV findContours function.
    # Note that there are three outputs, but we discard the first one.
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return False
    
    largest_area, largest_contour_index = findGreatesContour(contours)

    print(largest_area)
    
    center_x = 250 # this is where the camera thought the center was, should be 220 though
    
    largest_contour = contours[largest_contour_index]
    
    # returns the top left coordinate and the width
    x, y, w, h = cv2.boundingRect(largest_contour)
    rect_center = x + (w/2)
    
    # check to see if the box is in front of us
    if (rect_center > center_x - diff_center) and (rect_center < center_x + diff_center):
        front = True
    else: front = False 
    
    # will probbaly want to make this a lot larger
    # 100 is a 10 pixel by 10 pixel shape which is quite small considering the shape of the image
    if (largest_area > max_area) and front:
        return True
    else:
        return False


# Follow the setpoint sequence trajectory:
def adjust_position(cf, current_x, current_y):
    # NOTE Added current x to be able to move this based on 
    # new x positions aswell
    # always move to the right until at a certain y_position, then start to move
    # towards the left, oscillate between the two

    print('Adjusting position')

    min_y = -0.5
    max_y = 0.5
    left = False
    
    steps_per_meter = int(10)
    # Set the number here (the iterations of the for-loop) to the number of side steps.
    # You may choose to tune the number and size of the steps.
    
    if current_y <= min_y:
        left = True
    for i in range(3): 
        
        if left:
            current_y = current_y + 1.0/float(steps_per_meter)
        # figure out which direction to move
        else:
            current_y = current_y - 1.0/float(steps_per_meter) # minus moves to the right (at least thats what the drone was doing)
                                                            # plus moves in to the left in the image
        current_x = current_x + 1.0/float(steps_per_meter)
        
        position = [current_x, current_y, 1.0, 0.0]

        print('Setting position {}'.format(position))
        
        # not sure what this loop is doing
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    #cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed.
    # The message queue is not flushed before closing.
    time.sleep(0.1)
    return current_y, current_x

###############################################
##########            ADDED       #############
###############################################
def gap_move(cf, mask, current_x, current_y):
    # locate the gap and move through it
    raise NotImplementedError
    
###############################################
    
def move_forward(cf, current_x, current_y):
    print('Moving Forward')
    
    steps_per_meter = int(10)
    
    # Set the number here (the iterations of the for-loop) to the number of forward steps.
    # You may choose to tune the number and size of the steps.
    # perhaps want to check the countours after a certain number of steps?
    for i in range(3): 
        current_x = current_x + 1/float(steps_per_meter)
        
        position = [current_x, current_y, 1.0, 0]
        
        print(f"Setting Position {position}")
        
        # not sure what this loop is doing
        for i in range(10):
            cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
            time.sleep(0.1)

    #cf.commander.send_stop_setpoint()
    # Make sure that the last packet leaves before the link is closed.
    # The message queue is not flushed before closing.
    time.sleep(0.1)
    return current_x

def find_book(mask):
    """
    Determine if there is a book in the line of sight
    
    This should happen close to the end
    """
    
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    if not contours:
        return None
    
    largest_area, largest_contour_index = findGreatesContour(contours)

    largest_contour = contours[largest_contour_index]
    
    # returns the top left coordinate and the width
    x, y, w, h = cv2.boundingRect(largest_contour)
    rect_center = x + (w/2)
    
    if (w>h):
        center_x = x + (w/2)
        center_y = y + (h/2)
        return center_x, center_y, h
    
    else: return None

def move_to_book(cf, book_x, book_y, book_height, current_x, current_y):
    """
    Put the book into our center of view
    """
    
    center_x = 250
    image_height = 480
    
    if (np.abs(center_x -book_x) <= 10):
        print(f'{x_center} and book center: {book_center}')
        print('centered book')
        return False
    
    y_velocity = (center_x - book_x)/100
    command_y = current_y + 0.08*y_velocity
    
    x_velocity = (image_height-book_height)/100
    command_x = current_x +0.03*x_velocity
    
    position = [command_x, command_y, 1.0, 0.0]
    
    print(f"Moving Towards Book {position}")
        
    # only set the position once since we are continuously updating
    cf.commander.send_position_setpoint(position[0],
                                                position[1],
                                                position[2],
                                                position[3])
    time.sleep(0.1)
    
    return command_x, command_y


# Hover, descend, and stop all motion:
def hover_and_descend(cf):
    print('Descending:')
    # Hover at 1.0 meters:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, 1.0)
        time.sleep(0.1)
    # Descend:
    for y in range(10):
        cf.commander.send_hover_setpoint(0, 0, 0, (10 - y) / 25)
        time.sleep(0.1)
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
    return

In [13]:
#######################################################
#########           ADDED             #################
#######################################################
def threshold_mask(avg_mask, thresh = 0.8):
    """
    avg_mask: the average mask that was computed
    thresh: the threshold at which we condiser a pixel white or black
    
    remove noise by thresholding the average
    """
    
    thresh_mask = avg_mask <= threshold
    
    # make whole image white 
    avg_mask[:, :] = 255
    
    # if less than threshold make the pixels black (0)
    avg_mask[thresh_mask] = 0
    
    
    return avg_mask
    

## Test obstacle avoidance on the CrazyFlie ##

The following cell *will* fly the drone. Place the CrazyFlie in front of an obstacle in the netted area for testing. This cell will perform object detection and avoidance using the red filtering defined in the helper functions above.

In [7]:


# set the number of frames to average over
num_frames = 3
threshold = 0.8
frame_shape = (480, 440)

In [11]:
# Set the URI the Crazyflie will connect to
uri = f'radio://0/{group_number}/2M'

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

# Check that CrazyFlie devices are available:
if len(available) == 0:
    print('No Crazyflies found, cannot run example')
else:
    ## Ascent to hover; run the sequence; then descend from hover:
    # Use the CrazyFlie corresponding to team number:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        # Get the Crazyflie class instance:
        cf = scf.cf
        current_y = 0.0
        current_x = 0.0

        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        index = 0
        avg_mask = np.zeros(frame_shape)
        
        cap = cv2.VideoCapture(camera)
        while(cap.isOpened()):

            ret, frame = cap.read()
            
            # crop the frame: take 100 off each side x values
            frame = frame[:, 100:540]

            elapsed = time.time() - t
            if(elapsed > 5.0):
                print('Capturing.....')

                if ret:
                    #cv2.imshow('frame',frame)

                    if(ascended_bool==0):
                        set_PID_controller(cf)
                        ascend_and_hover(cf)
                        ascended_bool = 1
                    else:
                        #if index % num_frames != 0:
                        #    mask = get_mask(frame)
                        #    avg_mask += mask
                        #    index+=1
                        #    continue
                            
                        # compute the average by dividing avg_frame by num frames
                        # this happens outside the if and before the else if, it will be skipped by the continue
                        #avg_mask = avg_mask / num_frames
                        
                        # threshold the average mask
                        #thresh_mask = threshold_mask(avg_mask, threshold)
                        
                        # reset the avg mask to be 0 
                        #avg_mask = np.zeros(frame_shape)
                        
                        thresh_mask = get_mask(frame)
                        
                        if current_x <= 2.4:# check if there is an obstacle right in front of you                            
                            if(check_contours(thresh_mask, diff_center = 70)):
                                #NOTE: we will need to adjust check contours to account for if we are looking at
                                # an obstacle or if we are looking at the book
                                current_y, current_x = adjust_position(cf, current_x, current_y)
                            else:
                                # maybe change this to be move towards book? but dont want to risk moving into an obstacle
                                current_x = move_forward(cf, current_x, current_y)
                        else:
                            print('Finding Book')
                            
                            if find_book(thresh_mask) is None:
                                hover(cf)
                            else:
                                book_x, book_y, h = find_book(thresh_mask)
                                if (not move_to_book(cf, book_x, book_y, h, current_x, current_y)):
                                    break
                                
                                current_x, current_y = move_to_book(cf, book_x, book_y, h, current_x, current_y)
                            
                            
            
            # update this to exit the loop only when it is close enough to the book eventually
            #exit_loop = False
            #if exit_loop:
            #            break
            index += 1
            # adjust this to make it run longer
            if(elapsed > 600.0):
                print('time out')
                break
            
            cv2.imshow('mask', frame)
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()

        # Descend and stop all motion:
        hover_and_descend(cf)

print('Done!')

Scanning interfaces for Crazyflies...
Crazyflies found:
radio://0/9/2M
radio://0/14/2M
radio://0/9/2M
radio://0/14/2M
radio://0/6/2M
radio://0/9/2M
radio://0/14/2M
radio://0/9/2M
radio://0/14/2M
Capturing.....
Initializing PID Controller
Capturing.....
Checking image:
Moving Forward
Setting Position [0.1, 0.0, 1.0, 0]
Setting Position [0.2, 0.0, 1.0, 0]
Setting Position [0.30000000000000004, 0.0, 1.0, 0]
Capturing.....
Checking image:
16173.0
Adjusting position
Setting position [0.4, -0.1, 1.0, 0.0]
Setting position [0.5, -0.2, 1.0, 0.0]
Setting position [0.6, -0.30000000000000004, 1.0, 0.0]
Capturing.....
Checking image:
33370.5
Moving Forward
Setting Position [0.7, -0.30000000000000004, 1.0, 0]
Setting Position [0.7999999999999999, -0.30000000000000004, 1.0, 0]
Setting Position [0.8999999999999999, -0.30000000000000004, 1.0, 0]
Capturing.....
Checking image:
104.5
Moving Forward
Setting Position [0.9999999999999999, -0.30000000000000004, 1.0, 0]
Setting Position [1.0999999999999999, 

In [24]:
cap.release()